In [ ]:
import dotenv
dotenv.load_dotenv('../.env')

from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import MetadataMode

from projectgurukul.readers import CSVReader,RamayanaCSVReader, MahabharataCSVReader
import pandas as pd
import re

# Gita

In [14]:
def preprocess(row):
    row['chapter'], row['verse'] = row.verse_number.split(', ')
    return row


reader = CSVReader(text_columns=['verse','verse_in_sanskrit','translation_in_english','meaning_in_english'], metadata_columns=['chapter'], preprocess = preprocess)

documents = reader.load_data('../data/gita/data/bhagavad_gita.csv', extra_info={'souce': 'Bhagavad Gita'})

In [15]:
len(documents)

701

In [16]:
documents[0].metadata_template = "> {key}: {value}"
print(documents[0].get_content(metadata_mode = MetadataMode.ALL))

> chapter: Chapter 1
> souce: Bhagavad Gita

Verse 1

धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः |मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय ||1||

Dhritarashtra said: O Sanjay, after gathering on the holy field of Kurukshetra, and desiring to fight, what did my sons and the sons of Pandu do?

The two armies had gathered on the battlefield of Kurukshetra, well prepared to fight a war that was inevitable. Still, in this verse, King Dhritarashtra asked Sanjay, what his sons and his brother Pandu’s sons were doing on the battlefield? It was apparent that they would fight, then why did he ask such a question?The blind King Dhritarashtra’s fondness for his own sons had clouded his spiritual wisdom and deviated him from the path of virtue. He had usurped the kingdom of Hastinapur from the rightful heirs; the Pandavas, sons of his brother Pandu. Feeling guilty of the injustice he had done towards his nephews, his conscience worried him about the outcome of this battle.The words dhar

In [17]:
documents[1].get_metadata_str()

'chapter: Chapter 1\nsouce: Bhagavad Gita'

In [18]:
wikireader = WikipediaReader()
wikidocs = wikireader.load_data(['ayodhya ram temple'])


NameError: name 'WikipediaReader' is not defined

In [19]:
print( wikidocs[0].get_content()[:1500])

NameError: name 'wikidocs' is not defined

# Ramayana

In [30]:
df = pd.read_csv("../data/ramayana/data/yuddhakanda.csv").dropna(how = 'all')
df.tail()

,content,explanation
5204,एवमेतत्पुरावृत्तमाख्यानंभद्रमस्तुवः ।।6.131.12...,This way Ramayana is the ancient narration. Re...
5205,देवाश्चसर्वेतुष्यन्तिग्रहणाच्छ्रवणात्तथा ।।6.1...,"By grasping and listening to Ramayana, all god..."
5206,भक्त्यारामस्ययेचेमांसंसितामृषिणाकृताम् ।।6.131...,Residence in heaven is assured for those who t...
5207,कुटुम्बवृद्धिंधनधान्यवृद्धिंस्त्रियश्चमुख्यास्...,Listening to this great auspicious epic will r...
5208,आयुष्यमारोग्यकरंयशस्यंसौभ्रातृकंबुद्धिकरंशुभं ...,If the narrative of Ramayana is regularly hear...


In [31]:
docs = RamayanaCSVReader().load_data("../data/ramayana/data/balakanda.csv")

In [36]:
print(docs[0].get_content(MetadataMode.LLM))

sarga: 1

Ascetic Valmiki enquired of Narada, preeminent among the sages ever engaged in the practice of religious austerities or study of the Vedas and best among the eloquent.  ।।1.1.1।।

"Who in this world lives today endowed with excellent qualities, prowess, righteousness, gratitude, truthfulness and firmness in his vows? ।।1.1.2।।

Who is that one gifted with good conduct, given to the wellbeing of all living creatures, learned in the lore (knowledge of all things that is known), capable of doing things which others can not do and singularly handsome?   ।।1.1.3।।


Who (among men) is selfrestrained? Who has conquered anger? Who is endowed with brilliance and free from envy? Who is that when excited to wrath even the devatas, are afraid of (let alone foes)? ।।1.1.4।।

 O Maharshi, I intend to hear about such a man whom you are able to place? Indeed great is my curiosity". ।।1.1.5।।


Invited by Valmiki to take his seat Narada, knower of the three worlds heard him and said with del

# Mahabharata

In [22]:
import csv
import re
import pandas as pd
import random
MAHABHARATA_OUT_FILE = "../data/mahabharata/data/mahabharata_parsed.csv"

In [23]:
all_lines = [] 
with open('../data/mahabharata/raw/1-18 books combined.txt', 'r') as reader:
    all_lines = reader.readlines()
    

In [24]:

with open(MAHABHARATA_OUT_FILE, "w") as file:
    wr = csv.writer(file)
    wr.writerow(["parva", "chapter",  "chapter title" , "content"])
    current_parva = 0
    current_chapter = 0
    capture_current_chapter_title = False
    for line in all_lines:
        if line == '\n' or len(line.strip()) == 0:
            continue
        if capture_current_chapter_title:
            current_chapter_title = line.strip()
            capture_current_chapter_title = False
            continue
        if re.match('^\w+ Parva$\n', line):
            current_parva = line.strip()
            continue
        if line.startswith('Chapter') and line != 'Chapter Commentary\n':
            current_chapter = line.strip()
            capture_current_chapter_title = True
            continue
        wr.writerow([current_parva, current_chapter,  current_chapter_title , line.strip()])


In [25]:

with open(MAHABHARATA_OUT_FILE, 'r') as file:
    df = pd.read_csv(file).dropna(how="all").fillna("")
    print(dict(zip(*[iter(df['parva'].unique()), iter(range(1,df.size + 1))])))
    

{'Adi Parva': 1, 'Sabha Parva': 2, 'Vana Parva': 3, 'Virata Parva': 4, 'Udyoga Parva': 5, 'Bhisma Parva': 6, 'Drona Parva': 7, 'Karna Parva': 8, 'Salya Parva': 9, 'Sauptika Parva': 10, 'Stree Parva': 11, 'Shanti Parva': 12, 'Anushasana Parva': 13, 'Ashvamedha Parva': 14, 'Ashramvasika Parva': 15, 'Mausala Parva': 16, 'Mahaprasthanika Parva': 17}


In [26]:
df.head(3)

,parva,chapter,chapter title,content
0,Adi Parva,Chapter One,Maharaja Shantanu Marries the Celestial Ganga,According to the historical records of this ea...
1,Adi Parva,Chapter One,Maharaja Shantanu Marries the Celestial Ganga,"Once when Maharaja Shantanu, that bull among m..."
2,Adi Parva,Chapter One,Maharaja Shantanu Marries the Celestial Ganga,The beautiful apsara (celestial maiden) then s...


In [27]:
df_grouped = df.groupby(['parva', 'chapter', 'chapter title'],as_index=True).agg(lambda lst: "\n\n".join(lst)).reset_index()
df_grouped.head(3)

,parva,chapter,chapter title,content
0,Adi Parva,Chapter Eight,The Preceptor Drona,"Seeing the princes enter adolescence, Maharaja..."
1,Adi Parva,Chapter Eighteen,Arjuna Goes on Pilgrimage,After leaving Indraprastha in the dress of a m...
2,Adi Parva,Chapter Eleven,Tuition for Drona,Drona saw that all his students were now adept...


In [28]:
parva_id_map = dict(zip(*[iter(df['parva'].unique()), iter(range(1,df.size + 1))]))
print(parva_id_map)

{'Adi Parva': 1, 'Sabha Parva': 2, 'Vana Parva': 3, 'Virata Parva': 4, 'Udyoga Parva': 5, 'Bhisma Parva': 6, 'Drona Parva': 7, 'Karna Parva': 8, 'Salya Parva': 9, 'Sauptika Parva': 10, 'Stree Parva': 11, 'Shanti Parva': 12, 'Anushasana Parva': 13, 'Ashvamedha Parva': 14, 'Ashramvasika Parva': 15, 'Mausala Parva': 16, 'Mahaprasthanika Parva': 17}


In [29]:

documents = []
for i, row in df.iterrows():
            parva = row['parva']
            parva_id = parva_id_map[parva]
            chapter_id = row['chapter']
            chapter_title = row['chapter title']
            content = row['content']
            page_link = 118367 + i

            metadata = {
                'parva_id' : parva_id,
                'parva': parva,
                'chapter_id':chapter_id,
                'chapter_title':chapter_title,
                'page_link':page_link
            }

            documents.append((metadata, content))
print(documents[0])

({'parva_id': 1, 'parva': 'Adi Parva', 'chapter_id': 'Chapter One', 'chapter_title': 'Maharaja Shantanu Marries the Celestial Ganga', 'page_link': 118367}, 'According to the historical records of this earth, there once lived a King named Maharaja Shantanu, the son of Pratipa, who took his birth in the solar dynasty and was considered naradeva, the manifest representative of the Supreme Lord on earth. His fame and rule extended to all parts of the world. The qualities of self-control, liberality, forgiveness, intelligence, modesty, patience and power always resided this exalted emperor. His neck was marked with three lines like a conchshell, and his shoulders were broad. In prowess He resembled a maddened elephant. Above all these qualities, he was a devoted servant of Lord Vishnu, and therefore he was given the title, "King of kings".')


In [45]:

metadata = documents[random.randint(0,len(documents))][0]
print(metadata)

{'parva_id': 2, 'parva': 'Sabha Parva', 'chapter_id': 'Chapter Nine', 'chapter_title': 'The Gambling Match', 'page_link': 119076}


In [48]:
print(f"https://en.wikisource.org/wiki/The_Mahabharata/Book_{parva_id}:_{parva.replace(' ','_')}")

https://en.wikisource.org/wiki/The_Mahabharata/Book_17:_Mahaprasthanika_Parva


In [10]:
documents = MahabharataCSVReader().load_data(MAHABHARATA_OUT_FILE)
documents[0]

NameError: name 'MahabharataCSVReader' is not defined

In [9]:
documents[0].metadata

NameError: name 'documents' is not defined